In [2]:
from lib import *
dir_lookup = {"R": (1, 0), "L": (-1, 0), "U": (0, 1), "D": (0, -1)}

In [189]:
@level_ab(1, test=('1000\n2000\n3000\n\n4000\n\n5000\n6000\n\n7000\n8000\n9000\n\n10000', 24000, 45000), sep="\n\n")
def solve(num: List[str], level=0):
    chunks = [sum(map(int, line.split("\n"))) for line in num]
    return sum(list(sorted(chunks, reverse=True))[:3]) if level else max(chunks)

Part a already solved with same answer: 69912
Part b already solved with same answer: 208180


In [190]:
@level_ab(2, test=("""A Y\nB X\nC Z""", 15, 12), sep="\n", apply=lambda line: line.split(" "))
def solve(lines: List[str], level=0):
    total = 0
    for a, b in lines:
        a, b = ord(a) - ord('A'), ord(b) - ord('X')
        total += b * 3 + (a+b-1) % 3 + 1 if level else ((b-a+1)%3) * 3 + b + 1
    return total

Part a already solved with same answer: 11603
Part b already solved with same answer: 12725


In [191]:
def prio(c: str) -> int:
    return ord(c.lower()) - ord('a') + 1 + c.isupper() * 26

@level_ab(3, test=("""vJrwpWtwJgWrhcsFMMfFFhFp\njqHRNqRjqzjGDLGLrsFMfFZSrLrFZsSL\nPmmdzqPrVvPwwTWBwg\nwMqvLMZHhHMvwLHjbvcjnnSBnvTQFn\nttgJtRGJQctTZtZT\nCrZsJsPPZsGzwwsLwLmpwMDw""", 157, 70), sep="\n")
def solve(lines: List[str], level=0):
    total = 0
    if level:
        for i in range(0, len(lines)-1, 3):
            a, b, c = map(set, lines[i: i+3])
            total += prio(next(iter(a & b & c)))
    else:
        for line in lines:
            offset = len(line)//2
            a, b = set(line[:offset]), set(line[offset:])
            total += sum([prio(c) for c in a & b])
    return total

Part a already solved with same answer: 8176
Part b already solved with same answer: 2689


In [192]:
@level_ab(4, test=("""2-4,6-8\n2-3,4-5\n5-7,7-9\n2-8,3-7\n6-6,4-6\n2-6,4-8""", 2, 4), sep="\n",
          apply=lambda line: map(int, re.findall(r"(\d+)-(\d+),(\d+)-(\d+)", line)[0]))
def solve(lines: List[str], level=0, ):
    total = 0
    for a, b, c, d in lines:
        left, right = set(range(a, b+1)), set(range(c, d+1))
        total += bool(left & right) if level else left.issuperset(right) or right.issuperset(left)
    return total

Part a already solved with same answer: 584
Part b already solved with same answer: 933


In [193]:
@level_ab(5, test=('    [D]\n[N] [C]\n[Z] [M] [P]\n 1   2   3\n\nmove 1 from 2 to 1\nmove 3 from 1 to 3\nmove 2 from 2 to 1\nmove 1 from 1 to 2', "CMZ", "MCD"), sep="\n")
def solve(lines: List[str], level=0):
    stacks = defaultdict(list)
    lines = iter(lines)
    for line in lines:
        if line.startswith(" 1 "):
            next(lines)  # forward iterator by 1
            break
        for pos, dst in enumerate(line):
            if dst.isalpha():  # skip "[", "]", " "
                stacks[1+pos//4].insert(0, dst)
    for line in lines:
        _, amount, _, src, _, dst = line.split(" ")
        amount, src, dst = int(amount), int(src), int(dst)
        picked = [stacks[src].pop() for _ in range(amount)]
        stacks[dst].extend(reversed(picked) if level else picked)
    return "".join([stacks[i][-1] for i in sorted(stacks.keys())])

Part a already solved with same answer: TPGVQPFDH
Part b already solved with same answer: DMRDFRHHH


In [194]:
@level_ab(6, test=("""mjqjpqmgbljsphdztnvjfqwrcgsmlb""", 7, 19), sep="\n")
def solve(lines: List[str], level=0):
    offset = 4 + level * 10
    for line in lines:
        for i in range(len(line)-offset):
            if len(set(line[i:i+offset])) == offset:
                return i+offset

Part a already solved with same answer: 1134
Part b already solved with same answer: 2263


In [195]:
def dir_sum(dir_name: str, root: Dict[str, Dict], dirs_name_size: Dict[str, int]) -> int:
    """ recursively sum up sizes of subdirectories and add all to a global list """
    total_size = 0
    for file_name, file_dict_or_size in root.items():
        if type(file_dict_or_size) == dict and file_name != "..":
            total_size += dir_sum(dir_name + "/" + file_name, file_dict_or_size, dirs_name_size)
        elif type(file_dict_or_size) == int:
            total_size += file_dict_or_size
    dirs_name_size[dir_name] = total_size
    return total_size

@level_ab(7, test=('$ cd /\n$ ls\ndir a\n14848514 b.txt\n8504156 c.dat\ndir d\n$ cd a\n$ ls\ndir e\n29116 f\n2557 g\n62596 h.lst\n$ cd e\n$ ls\n584 i\n$ cd ..\n$ cd ..\n$ cd d\n$ ls\n4060174 j\n8033020 d.log\n5626152 d.ext\n7214296 k', 95437, 24933642), sep="\n")
def solve(lines: List[str], level=0):
    root = curr = dict()
    for line in lines:
        a, b, *c = line.split(" ")
        if a == "$":
            if b == "cd":  # just ignore ls case
                c = c[0]
                curr = root if c == "/" else curr[c]
        else:
            curr[b] = {"..": curr} if a == "dir" else int(a)
    total = dir_sum("", root, dirs_name_size := dict())
    level_filter = [lambda s: s < 100_000, lambda s: s > total - 40000000][level]
    sizes = sorted(filter(level_filter, dirs_name_size.values()))
    return min(sizes) if level else sum(sizes)

Part a already solved with same answer: 1989474
Part b already solved with same answer: 1111607


In [196]:
@level_ab(8, test=('30373\n25512\n65332\n33549\n35390', 21, 8), sep="\n")
def solve(lines: List[str], level=0):
    f = np.array([[int(c) for c in line] for line in lines])
    vis, score = np.zeros_like(f), np.ones_like(f, dtype=int)
    for _ in range(4):  # repeat with 4 rotations
        for (i, j), val in np.ndenumerate(f):
            if level:  # walk down 1, 2, 3... steps
                for d in range(0, len(f)-j):
                    if d == 0:
                        continue
                    if f[i, j+d] >= f[i, j]:
                        break
                score[i, j] *= d  # noqa
            else:
                if j == 0 or max(f[i, :j]) < f[i, j]:
                    vis[i, j] = True
        f, vis, score = np.rot90(f), np.rot90(vis), np.rot90(score)
    return score.max() if level else vis.sum()

Part a already solved with same answer: 1662
Part b already solved with same answer: 537600


In [197]:
@level_ab(9, test=('R 4\nU 4\nL 3\nD 1\nR 4\nD 1\nL 5\nR 2', 13, 1), sep="\n")
def solve(lines: List[str], level=0):
    snake = np.zeros((10 if level else 2, 2), dtype=int)
    visited = {tuple(snake[0])}
    for line in lines:
        dir_letter, amount = line.split(" ")
        for _ in range(int(amount)):
            snake[0] += dir_lookup[dir_letter]  # move head
            for i in range(1, len(snake)):
                dist = snake[i-1] - snake[i]
                if abs(dist).max() > 1:  # pull head if x or y dist > 1
                    snake[i] += np.sign(dist)  # pull maximally 1/1, -1/1, 1/0...
                visited.add(tuple(snake[-1]))
    return len(visited)

Part a already solved with same answer: 6486
Part b already solved with same answer: 2678


In [198]:
@level_ab(10, test=('addx 15\naddx -11\naddx 6\naddx -3\naddx 5\naddx -1\naddx -8\naddx 13\naddx 4\nnoop\naddx -1\naddx 5\naddx -1\naddx 5\naddx -1\naddx 5\naddx -1\naddx 5\naddx -1\naddx -35\naddx 1\naddx 24\naddx -19\naddx 1\naddx 16\naddx -11\nnoop\nnoop\naddx 21\naddx -15\nnoop\nnoop\naddx -3\naddx 9\naddx 1\naddx -3\naddx 8\naddx 1\naddx 5\nnoop\nnoop\nnoop\nnoop\nnoop\naddx -36\nnoop\naddx 1\naddx 7\nnoop\nnoop\nnoop\naddx 2\naddx 6\nnoop\nnoop\nnoop\nnoop\nnoop\naddx 1\nnoop\nnoop\naddx 7\naddx 1\nnoop\naddx -13\naddx 13\naddx 7\nnoop\naddx 1\naddx -33\nnoop\nnoop\nnoop\naddx 2\nnoop\nnoop\nnoop\naddx 8\nnoop\naddx -1\naddx 2\naddx 1\nnoop\naddx 17\naddx -9\naddx 1\naddx 1\naddx -3\naddx 11\nnoop\nnoop\naddx 1\nnoop\naddx 1\nnoop\nnoop\naddx -13\naddx -19\naddx 1\naddx 3\naddx 26\naddx -30\naddx 12\naddx -1\naddx 3\naddx 1\nnoop\nnoop\nnoop\naddx -9\naddx 18\naddx 1\naddx 2\nnoop\nnoop\naddx 9\nnoop\nnoop\nnoop\naddx -1\naddx 2\naddx -37\naddx 1\naddx 3\nnoop\naddx 15\naddx -21\naddx 22\naddx -6\naddx 1\nnoop\naddx 2\naddx 1\nnoop\naddx -10\nnoop\nnoop\naddx 20\naddx 1\naddx 2\naddx 2\naddx -6\naddx -11\nnoop\nnoop\nnoop', 13140, "PGHFGLUG"), sep="\n")
def solve(lines: List[str], level=0):
    states, x = [1], 1
    for line in lines:
        states.append(x)
        if line.startswith("addx"):
            x += int(line[5:])
            states.append(x)
    if level:
        for t, v in enumerate(states):
            print("#" if ((v-t+1)%40)//3==0 else " ", end="\n" if t%40 == 39 else "")
        print()
        return "PGHFGLUG"
    else:
        return sum([states[i-1]*i for i in (20, 60, 100, 140, 180, 220)])

Part a already solved with same answer: 15260
##  ##  ##  ##  ##  ##  ##  ##  ##  ##  
###   ###   ###   ###   ###   ###   ### 
####    ####    ####    ####    ####    
#####     #####     #####     #####     
######      ######      ######      ####
#######       #######       #######     
 
###   ##  #  # ####  ##  #    #  #  ####
#  # #  # #  # #    #  # #    #  # #  ##
#  # #    #### ###  #    #    #  # #   #
###  # ## #  # #    # ## #    #  # # ###
#    #  # #  # #    #  # #    #  # #  # 
#     ### #  # #     ### ####  ##   ### 
 
Part b already solved with same answer: PGHFGLUG


In [199]:
class Monkey:
    def __init__(self, chunk: str):
        self.residue = 0  # equivalence class for numbers
        self.cnt = 0
        self.next_items = []

        self.id, self.items, self.op, self.num, self.divisor, self.if_true, self.if_false = re.match(r"""Monkey (\d+):
  Starting items: (\d+(?:, \d+)*)
  Operation: new = old (.) (.+)
  Test: divisible by (\d+)
    If true: throw to monkey (\d+)
    If false: throw to monkey (\d+)""", chunk).groups()
        self.id, self.divisor, self.if_true, self.if_false = int(self.id), int(self.divisor), int(self.if_true), int(self.if_false)
        self.items = [int(i) for i in self.items.split(", ")]

        if self.num == "old":
            self.op = lambda x: x**2
        else:
            self.num = int(self.num)
            if "*" in self.op:
                self.op = lambda x: x * self.num
            elif "+" in self.op:
                self.op = lambda x: x + self.num

    def process(self, monkeys):
        self.cnt += len(self.items)
        for i in self.items:
            i = self.op(i)
            if self.residue:
                i = i % self.residue
            else:
                i //= 3
            target = self.if_true if i % self.divisor == 0 else self.if_false
            monkeys[target].items.append(i)
        self.items = []


@level_ab(11, test=('Monkey 0:\n  Starting items: 79, 98\n  Operation: new = old * 19\n  Test: divisible by 23\n    If true: throw to monkey 2\n    If false: throw to monkey 3\n\nMonkey 1:\n  Starting items: 54, 65, 75, 74\n  Operation: new = old + 6\n  Test: divisible by 19\n    If true: throw to monkey 2\n    If false: throw to monkey 0\n\nMonkey 2:\n  Starting items: 79, 60, 97\n  Operation: new = old * old\n  Test: divisible by 13\n    If true: throw to monkey 1\n    If false: throw to monkey 3\n\nMonkey 3:\n  Starting items: 74\n  Operation: new = old + 3\n  Test: divisible by 17\n    If true: throw to monkey 0\n    If false: throw to monkey 1', 10605, 2713310158), sep="\n\n")
def solve(chunks: List[str], level=0):
    monkeys = [Monkey(chunk) for chunk in chunks]
    if level:
        divisors = np.lcm.reduce([m.divisor for m in monkeys])  # noqa
        for m in monkeys:
            m.residue = divisors
    for i in range(10000 if level else 20):
        for m in monkeys:
            m.process(monkeys)
    return np.prod(sorted([m.cnt for m in monkeys])[-2:])

Part a already solved with same answer: 110264
Part b already solved with same answer: 23612457316


In [200]:
def coord_in_arr(coord: Tuple[int, int], arr: np.ndarray) -> bool:
    return all(0 <= pos < dim_len for pos, dim_len in zip(coord, arr.shape))

def pos_of_value(arr: np.ndarray, value: Any):
    return next(zip(*np.where(arr==value)))

@level_ab(12, test=('Sabqponm\nabcryxxl\naccszExk\nacctuvwj\nabdefghi', 31, 29), sep="\n")
def solve(lines: List[str], level=0):
    f = np.array([[ord(c) for c in line] for line in lines])
    start = pos_of_value(f, ord("S"))
    end = pos_of_value(f, ord("E"))
    f[start], f[end] = ord('a'), ord('z')

    g = nx.DiGraph()
    for pos, curr in np.ndenumerate(f):
        for x, y in dir_lookup.values():
            e = pos[0] + x, pos[1] + y
            if coord_in_arr(e, f) and curr <= f[e] + 1:
                g.add_edge(e, pos)
    if level:
        apos = zip(*np.where(f==ord('a')))
        return min(nx.shortest_path_length(g, start, end) for start in apos if nx.has_path(g, start, end))
    else:
        return nx.shortest_path_length(g, start, end)

Part a already solved with same answer: 472
Part b already solved with same answer: 465


In [10]:
def compare(a: int | List, b: int | List) -> Optional[bool]:
    if isinstance(a, int) and isinstance(b, int):
        if a == b:  # Otherwise, the inputs are the same integer
            return None  # continue checking the next part of the input.
        return a < b  # both values are integers, the lower integer should come first
    elif isinstance(a, int):  # If exactly one value is an integer, convert the integer to a
        return compare([a], b)  # list which contains that integer as its only value, then retry the comparison
    elif isinstance(b, int):
        return compare(a, [b])
    elif isinstance(a, list) and isinstance(b, list):  # If both values are lists
        for i in range(len(a)):  # compare the first value of each list, then the second value, and so on.
            if i == len(b):
                return False  # If the right list runs out of items first, the inputs are not in the right order
            if (res := compare(a[i], b[i])) is not None:
                return res
        if len(a) == len(b):  # If the lists are the same length and no comparison makes a decision about the order
            return None  # continue checking the next part of the input
        return True  # If the left list runs out of items first, the inputs are in the right order


def bubble_sort(data):
    for i in range(len(data)):
        for j in range(i + 1, len(data)):
            if not compare(data[i], data[j]):
                data[i], data[j] = data[j], data[i]
    return data

@level_ab(13, test=('[1,1,3,1,1]\n[1,1,5,1,1]\n\n[[1],[2,3,4]]\n[[1],4]\n\n[9]\n[[8,7,6]]\n\n[[4,4],4,4]\n[[4,4],4,4,4]\n\n[7,7,7,7]\n[7,7,7]\n\n[]\n[3]\n\n[[[]]]\n[[]]\n\n[1,[2,[3,[4,[5,6,7]]]],8,9]\n[1,[2,[3,[4,[5,6,0]]]],8,9]', 13, 140), sep="\n\n")
def solve(lines: List[str], level=0):
    valid_chunks = []
    if level:
        data = [line.replace("\n", ",") for line in lines]
        data = list(eval(",".join(data))) + [[2], [6]]
        # bubble sort
        bubble_sort(data)
        return (data.index([2])+1) * (data.index([6])+1)
    else:
        for i, chunk in enumerate(lines):
            data = eval(chunk.replace("\n", ","))
            if compare(*data):
                valid_chunks.append(i+1)
    return sum(valid_chunks)

Part a already solved with same answer: 6272
Part b already solved with same answer: 22288


In [ ]:
def drop_sand(grid: np.array, x: int = 500, y: int = 0) -> bool:
    for y in range(y, grid.shape[1]-1):
        if grid[x, y] == "." and grid[x, y + 1] in "#o":
            if grid[x - 1, y + 1] == ".":  # try left
                return drop_sand(grid, x - 1, y + 1)
            elif grid[x + 1, y + 1] == ".":  # try right
                return drop_sand(grid, x + 1, y + 1)
            else:  # straight drop
                grid[x, y] = "o"
                return True
    return False  # dropped out of the grid


@level_ab(14, test=("""498,4 -> 498,6 -> 496,6
503,4 -> 502,4 -> 502,9 -> 494,9""", 24, 93), sep="\n")
def solve(lines: List[str], level=0):
    field = np.full((1000, 200), ".")
    highest_y = 0
    for line in lines:
        line_parts = [[int(s) for s in p.split(",")] for p in line.split(" -> ")]
        for (x, y), (xn, yn) in zip(line_parts, line_parts[1:]):
            field[x, y] = field[xn, yn] = "#"
            if x == xn:
                field[xn, min(yn, y):max(yn, y)] = "#"
            else:
                field[min(xn, x):max(xn, x), yn] = "#"
            highest_y = max(yn, highest_y)
    if level:
        field[:, highest_y+2] = "#"  # draw "floor"
    while drop_sand(field, 500, 0) and field[500, 0] == ".":
        pass
    return (field == "o").sum()


In [ ]:
@level_ab(15, test=("""498,4 -> 498,6 -> 496,6
503,4 -> 502,4 -> 502,9 -> 494,9""", 24, 93), sep="\n")
def solve(lines: List[str], level=0):
    return ""